# Modeling Disjunctions through the Strip Packing Problem

**Prepared by:** Prof. Alexander Dowling (adowling@nd.edu), [Hailey Lynch](https://github.com/hglynch) (hlynch@nd.edu, 2023)

## Introduction and Learning Objectives

This notebook illustrates a more complicated example of generalized disjunctive programs. Students will practice applying concepts of Logical Modeling and Generalized Disjunctive Programs to the Strip Packing Problem. Critical thinking discussion questions will be included to connect concepts from CBE 60499.

See the following notebook for a primer on Logical Modeling and Generalized Disjunctive Programs: **Insert Reactor Notebook when available -- This is the other notebook I am working on and has information in it that relates to this, it will be linked once they are finalized**

## Import Modules

In [ ]:
# Imports
import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py"
    import helper
    helper.install_idaes()
    helper.install_ipopt()
    # helper.install_cbc()
    helper.download_figures(['strip_packing.png'])

--2023-05-04 18:16:25--  https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7171 (7.0K) [text/plain]
Saving to: ‘helper.py’

helper.py           100%[===================>]   7.00K  --.-KB/s    in 0s      

2023-05-04 18:16:26 (65.2 MB/s) - ‘helper.py’ saved [7171/7171]

Installing idaes via pip...
idaes was successfully installed
Running idaes get-extensions to install Ipopt, k_aug, and more...
Ipopt 3.13.2 (x86_64-pc-linux-gnu), ASL(20190605)

[K_AUG] 0.1.0, Part of the IDAES PSE framework
Please visit https://idaes.org/ (x86_64-pc-linux-gnu), ASL(20190605)

Couenne 0.5.8 -- an Open-Source solver for Mixed Integer Nonlinear Optimization
Mailing list: couenne@list.coin-or.org
Instructions

In [ ]:
milp_solver = 'cbc'

## Pyomo.GDP: Strip Packing Problem

We will be looking at the Strip Packing Problem using Pyomo.GDP as an example for modeling disjunctions.

> Aldo Vecchietti (1) and Ignacio Grossman (2) <br> (1) INGAR – Instituto de Desarrollo y Diseño – CONICET – UTN, Avellaneda 3657 – Santa Fe ‐ Argentina <br> (2) Carnegie Mellon University, 5000 Forbes Av. ‐ Pittsburgh, PA ‐ USA

| $$\text{Additional Information}$$       | $$\text{Link}$$        |
| ----------- | ----------- |
| $$\text{Reference}$$   |  https://www.minlp.org/library/problem/index.php?i=121&lib=GDP       |
| $$\text{Description}$$ | https://www.minlp.org/problems/ver/156/over/Strip-Packing-Overview.pdf        |
| $$\text{Results}$$    | https://www.sciencedirect.com/science/article/pii/S0098135405000992

### Problem Statement

$ \text{Min} \ lt \tag{1}$ 

$ \text{s.t.} \ lt \geq x_{i} + L_{i} \ \ \forall i \in N \tag{2}$

\begin{equation}
  \begin{bmatrix}
  Y_{ij}^{1} \\
  x_{i} + L_{i} \leq x_{j}
  \end{bmatrix}
 \lor
  \begin{bmatrix}
  Y_{ij}^{2} \\
  x_{j} + L_{j} \leq x_{i}
  \end{bmatrix}
 \lor 
  \begin{bmatrix}
  Y_{ij}^{3} \\
  y_{i} - H_{i} \geq y_{j}
  \end{bmatrix}
 \lor
  \begin{bmatrix}
  Y_{ij}^{4} \\
  y_{j} - H_{j} \geq y_{i}
  \end{bmatrix} \tag{3}
\end{equation} 

\begin{equation}
  x_{i} \leq UB_{i} - L_{i} \ \ \forall i \in N \tag{4}
\end{equation}

\begin{equation}
  H_{i} \leq y_{i} \leq W \ \ \forall i \in N \tag{5}
\end{equation}

\begin{equation}
 lt, x_{i}, y_{i} \in \mathbb{R}_{+}^{1}, Y_{ij}^{1}, Y_{ij}^{2}, Y_{ij}^{3}, Y_{ij}^{4} \in \text{\{True, False}\} \ \ \forall i, j \in N, i < j
\end{equation}

* The objective in this problem consists of minimizing the length of the strip $lt$ (1) and (2) by representing every rectangle by its coordinates in the $(x,y)$ space such that no overlap occurs between rectangles. 
* Thus, every rectangle $i \in N$ has length $L_{i}$, height $H_{i}$, and coordinates $(x_{i}, y_{i})$, where the point of reference corresponds to the upper left corner of every rectangle.
* By constraining every pair of rectangels $(i,j)$ where $(i,j \in N, i<j)$ that no overlap occurs, we obtain a series of disjunctions with four disjuncts each, where each disjunct represents the position of rectangle $i$ in relation to rectangle $j$ (3). 
* Note that the y-coordinate of every rectange is bounded from above by the fixed width of the strip $W$ (5), and that the upper bound $UB_{i}$, which in a best case scenario would correspond to the optimal value of $lt$, is obtained using a bottom-left rectangle-placing heurisitc and serves as an upper bound for the x-coordinate of every rectangle (4).

### Define model in Pyomo with GDP

First we will define the model for the Strip Packing Problem in Pyomo. 

In [ ]:
'''
Instead of using
# import pyomo.environ as pyo
We can import specific functions/objects
'''
from pyomo.environ import (ConcreteModel, NonNegativeReals, Objective, Param,
                           Set, SolverFactory, TransformationFactory, Var, value)

# Strip-packing example from http://minlp.org/library/lib.php?lib=GDP

# This model packs a set of rectangles without rotation or overlap within a
# strip of a given width, minimizing the length of the strip.

def create_model():
    '''
    Build the strip packing problem model.

    Return:
    model: Pyomo model
    
    '''
    
    ## Model
    model = ConcreteModel(name="Rectangles strip packing")
    
    ## Sets
    model.rectangles = Set(ordered=True, initialize=[0, 1, 2, 3, 4, 5, 6, 7])

    ## Parameters
    # Width and length of each rectangle
    model.rect_width = Param(
        model.rectangles, initialize={0: 3, 1: 3, 2: 2, 3: 2, 4: 3, 5: 5,
                                      6: 7, 7: 7})
    
    # Parameter indexed by each rectangle
    # Length means height
    model.rect_length = Param(
        model.rectangles, initialize={0: 4, 1: 3, 2: 2, 3: 2, 4: 3, 5: 3,
                                      6: 4, 7: 4})

    model.strip_width = Param(
        initialize=10, doc="Available width of the strip")

    # Upperbound on length (default is sum of lengths of rectangles)
    model.max_length = Param(
        initialize=sum(model.rect_length[i] for i in model.rectangles),
        doc="Maximum length of the strip (if all rectangles were arranged "
        "lengthwise)")

    ## Variables
    # x (length) and y (width) coordinates of each of the rectangles
    model.x = Var(model.rectangles,
                  bounds=(0, model.max_length),
                  doc="Rectangle corner x-position (position across length)")

    # Width bounds
    def w_bounds(m, i):
        return (0, m.strip_width - m.rect_width[i])
    model.y = Var(model.rectangles,
                  bounds=w_bounds,
                  doc="Rectangle corner y-position (position down width)")
    # String length
    model.strip_length = Var(
        within=NonNegativeReals, doc="Length of strip required.")

    # Rectangle conflicts
    def rec_pairs_filter(model, i, j):
        return i < j
    model.overlap_pairs = Set(
        initialize=model.rectangles * model.rectangles,
        dimen=2, filter=rec_pairs_filter,
        doc="Set of possible rectangle conflicts")

    ## Constraints
    @model.Constraint(model.rectangles)
    def strip_ends_after_last_rec(model, i):
        return model.strip_length >= model.x[i] + model.rect_length[i]

    ## Objective
    model.total_length = Objective(expr=model.strip_length,
                                   doc="Minimize length")

    
    ## Insert the no-overlap disjunctions here!
    
    ### BEGIN SOLUTION
    @model.Disjunction(
        model.overlap_pairs,
        doc="Make sure that none of the rectangles on the strip overlap in "
        "either the x or y dimensions.")
    def no_overlap(m, i, j):
        return [
            m.x[i] + m.rect_length[i] <= m.x[j],
            m.x[j] + m.rect_length[j] <= m.x[i],
            m.y[i] + m.rect_width[i] <= m.y[j],
            m.y[j] + m.rect_width[j] <= m.y[i],
        ]

    ### END SOLUTION

    return model

### Transform and Solve with Big M Relaxation

Now we will create the model and use Big-M Relaxation.

### Big-M Implementation in Pyomo

In [ ]:
# Creating the model
model = create_model()

Next, let's transform the model. The updated model is really big, so we will not print it. This is because the transformation factory replaced the disjunctions with many Big-M constraints.

In [ ]:
# Applying Big-M relaxation to the model
### BEGIN SOLUTION
TransformationFactory('gdp.bigm').apply_to(model)
### END SOLUTION

Finally, we'll solve the model and examine the solution.

In [ ]:
# Solve and print the solution
SolverFactory(milp_solver).solve(model, tee=True)
for i in model.rectangles:
    print("Rectangle %s: (%s, %s)" % (i, value(model.x[i]), value(model.y[i])))
model.total_length.display()

Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: Feb  3 2023 

command line - /content/cbc -printingOptions all -import /tmp/tmpyvqxaz49.pyomo.lp -stat=1 -solve -solu /tmp/tmpyvqxaz49.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 148 (-1) rows, 129 (-1) columns and 464 (-1) elements
Statistics for presolved model
Original problem has 112 integers (112 of which binary)
Presolved problem has 112 integers (112 of which binary)
==== 128 zero objective 2 different
128 variables have objective of 0
1 variables have objective of 1
==== absolute objective values 2 different
128 variables have objective of 0
1 variables have objective of 1
==== for integers 112 zero objective 1 different
112 variables have objective of 0
==== for integers absolute objective values 1 different
112 variables have objective of 0
===== end objective counts


Problem has 148 rows, 129 columns (1 with objective) and 464 elements
Column breakdown:
1 of type 

### Transform and Solve with Convex Hull Relaxation

We will repeat the procedure above using Convex Hull Relaxation.

### Convex Hull Implementation in Pyomo

In [ ]:
# Creating the model
model = create_model()

# Applying convex hull relaxation to the model
### BEGIN SOLUTION
TransformationFactory('gdp.hull').apply_to(model)
### END SOLUTION

# Solve and print the solution
SolverFactory(milp_solver).solve(model, tee=True)
for i in model.rectangles:
    print("Rectangle %s: (%s, %s)" % (i, value(model.x[i]), value(model.y[i])))
model.total_length.display()

Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: Feb  3 2023 

command line - /content/cbc -printingOptions all -import /tmp/tmp5_k8jzkf.pyomo.lp -stat=1 -solve -solu /tmp/tmp5_k8jzkf.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 484 (-113) rows, 353 (-113) columns and 1696 (-1) elements
Statistics for presolved model
Original problem has 112 integers (112 of which binary)
Presolved problem has 112 integers (112 of which binary)
==== 352 zero objective 2 different
352 variables have objective of 0
1 variables have objective of 1
==== absolute objective values 2 different
352 variables have objective of 0
1 variables have objective of 1
==== for integers 112 zero objective 1 different
112 variables have objective of 0
==== for integers absolute objective values 1 different
112 variables have objective of 0
===== end objective counts


Problem has 484 rows, 353 columns (1 with objective) and 1696 elements
Column breakdown:
1 of

## Discussion Questions

1. What is the advantage of using the decorator notation for optimization problems?
2. How do disjunctions affect a Degree of Freedom Analysis?
3. Why is it necessary to make separate variables for length and width?
4. Compare the outputs of Big-M and Convex Hull. How are they different (if at all)?

In [ ]:
# Discussion Questions
### BEGIN SOLUTION
'''
Answers:
1. Helpful for declaring components in Pyomo.
2. Disjunctions change the number of constraints.
3. The coordinates for length and width differ.
4. It gives the same optimal solution, but convex hull requires more iterations.
'''
### END SOLUTION

'\nAnswers:\n1. Helpful for declaring components in Pyomo.\n2. Disjunctions change the number of constraints.\n3. The coordinates for length and width differ.\n4. It gives the same optimal solution, but convex hull requires more iterations.\n'